In [1]:
import os
import pandas as pd
import numpy as np
import loompy as lp
#import crick
import math
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import tdigest
from tdigest import TDigest
import pickle


hey


# CSV to LOOM

In [ ]:
# First perform on head, then the clean CSV

## 1. Confirming the dimensions

In [ ]:
df = pd.read_csv('5_sorted_head.csv')

# Extract the gene expression values from columns 23 onwards
gene_expression_df = df.iloc[:, 16:]
gene_expression_data = gene_expression_df.values.T

print("Shape of gene_expression_data:", gene_expression_data.shape)
#print("Length of gene_expression_df.columns[16:]:", len(gene_expression_df.columns))


## 2. Dimension diagnostic test (Not Neccessary)

In [ ]:
# Not neccessary

df = pd.read_csv('5_sorted_head.csv')

# Select only the columns you're interested in (from index 16 onwards)
gene_expression_df = df.iloc[:, 16:]

# 1. Check for non-numeric data
non_numeric_cols = gene_expression_df.select_dtypes(exclude=['number']).columns
print("Columns with non-numeric data:", non_numeric_cols)

# 2. Check for missing data
missing_data_cols = gene_expression_df.columns[gene_expression_df.isnull().any()]
print("Columns with missing data:", missing_data_cols)

# 3. Check for columns with all identical values
single_value_cols = gene_expression_df.columns[gene_expression_df.nunique() == 1]
print("Columns with single unique value:", single_value_cols)

# Add any other checks you find relevant


## 3. Printing samples attribute cols

In [ ]:
df = pd.read_csv("clean_head.csv")

rows = df.shape[0]
columns = df.shape[1] 
print(f"rows = {rows}")
print(f"columns = {columns}")
print()

print("samples' attributes:")
print(df.columns[:17])

## 4. Main operation (run separately)

In [ ]:
import pandas as pd
import numpy as np
import loompy as lp


df = pd.read_csv('5_sorted_head.csv')

# Extract the cell attributes from columns 0 to 22 (including)
cell_attributes_df = df.iloc[:, :16]

# Extract the gene expression values from columns 23 onwards
gene_expression_df = df.iloc[:, 16:]
gene_expression_df = gene_expression_df.replace([np.nan, np.inf, -np.inf], -1)


# Transpose the gene expression DataFrame to have genes as rows and cells as columns
gene_expression_data = gene_expression_df.values.T

gene_expression_data = gene_expression_data.astype(float)


# Convert gene names to a numpy array
row_attrs = {"gene": df.columns[16:].astype('str').tolist()}


# Create the col_attrs dictionary containing all cell attributes
col_attrs = {}
for i in range(len(cell_attributes_df)):
    cell_attributes = cell_attributes_df.iloc[i].to_dict()
    for attr, value in cell_attributes.items():
        col_attrs.setdefault(attr, []).append(value)

col_attrs = {attr: np.array(value) for attr, value in col_attrs.items()}

        
# Create the .loom file for gene expression data
lp.create('16_head.loom', gene_expression_data, row_attrs=row_attrs, col_attrs=col_attrs)

# Checking loom file

## 1. Overview

In [ ]:
with lp.connect('16_head.loom') as ds:
    # Get the shape of the matrix
    matrix_shape = ds.shape

    # Get the row names (gene names)
    row_names = ds.ra.gene

    # Get the column names (cell attributes)
    col_names = ds.ca.keys()

    # Print the matrix shape
    print("Matrix Shape:", matrix_shape)

    # Print the row names (gene names)
    print("Row Attributes (Gene Names):")
    print(row_names)
    print()

    # Print the column names (cell attributes)
    print("Column Names (Cell Attributes):")
    print(col_names)
    print()

    # Print the first 100 rows and columns along with the data
    print("First 20 rows and columns:")
    print(ds[:10, :10])

## 2. Attributes of first col

In [ ]:
with lp.connect('16_head.loom') as ds:
    # Get the attributes and their values for the first column (cell)
    cell_attributes = ds.ca.items()

    # Print the attributes and their values for the first column (cell)
    print("Attributes and Values for the First Column (Cell):")
    print()
    for attr, values in cell_attributes:
        print(f"{attr}: {values[0]}")

## 3. How many 0 & -1 in loom file

In [ ]:
with lp.connect("16_head.loom") as ds:
    # Count the occurrences of -1 in the entire dataset
    num_negative_ones = (ds[:, :] == 0).sum()

print("Number of 0 values in the whole dataset:", num_negative_ones)

## 4. Checkin genes with 1 unique value

In [ ]:
with lp.connect("16_head.loom") as ds:
    # Get the gene expression data
    gene_expression_data = ds[:, :]
    
    # Calculate the number of unique values in each row (gene)
    unique_counts_row = np.apply_along_axis(lambda x: len(np.unique(x)), axis=1, arr=gene_expression_data)
    unique_counts_col = np.apply_along_axis(lambda x: len(np.unique(x)), axis=0, arr=gene_expression_data)
    
    # Find the indices of rows with only one unique value
    #rows_with_one_unique_value = np.where(unique_counts == 1)[0]
    rows_with_one_unique_value = np.where(unique_counts_row == 1)[0]
    columns_with_one_unique_value = np.where(unique_counts_col == 1)[0]
    
    
    # Print the row indices with only one unique value
    print("Rows (gene expressions) with only one unique value:")
    print(rows_with_one_unique_value)
    print(len(rows_with_one_unique_value))
    print()
    
    print("Colomns (samples) with only one unique value:")
    print(columns_with_one_unique_value)


## 5. Histogram of genes with 1 uniue value (needs indices)

In [ ]:
# Only if there is any genes with 1 unique value

with lp.connect("3_head.loom") as ds:
    # Get the gene expression data for the selected rows
    gene_expression_data = ds[12328:13305, :]
    
    # Flatten the data to create a 1D array of all values in the selected rows
    flattened_data = gene_expression_data.flatten()
    
    # Filter out -1 values from the array
    filtered_data = flattened_data[flattened_data != -1]
    
    # Calculate the distribution of unique values and their counts
    unique_values, counts = np.unique(filtered_data, return_counts=True)
    
    # Plot the histogram using Seaborn
    sns.histplot(filtered_data)
    plt.title("Genes with one unique expression value in mcf7 dataset")
    plt.xlabel("Unique Values")
    plt.ylabel("Counts")
    plt.xlim(-10,5)
    plt.show()
    plt.savefig("cells_values_mcf7.png", dpi=450)

## 6. Masking genes (add attr) with one unique value (zeros)

In [ ]:
import loompy as lp
import numpy as np

input_loom_file = "16_head.loom"  # Replace with your loom file path
chunk_size = 1000  # Number of rows to read at a time

# Initialize an empty array for the new attribute
new_attribute = np.zeros(0, dtype=int)

# Connect to the loom file
with lp.connect(input_loom_file) as ds:
    n_rows = ds.shape[0]
    
    for (ix, selection, view) in ds.scan(axis=0, batch_size=chunk_size):
        chunk_data = view[:, :]
        
        # Temporary array to store attribute values for this chunk
        temp_attribute = np.zeros(chunk_data.shape[0], dtype=int)
        
        # Loop through each row of the chunk and count unique values
        for i in range(chunk_data.shape[0]):
            row_data = chunk_data[i, :]
            unique_values = np.unique(row_data)
            
            # Check if the row has only one unique value
            if len(unique_values) == 1:
                temp_attribute[i] = 0  # Set to 0 if only one unique value
            else:
                temp_attribute[i] = 1  # Set to 1 otherwise
                
        # Append the temporary array to the new attribute array
        new_attribute = np.concatenate((new_attribute, temp_attribute))
        
    # Add the new attribute to the loom file
    ds.ra['UniqueValueIndicator'] = new_attribute


## 7. Checking the new attribute

In [ ]:
with lp.connect("16_head.loom") as ds:
    unique_value_indicator = ds.ra["UniqueValueIndicator"]
    count_zeros = np.sum(unique_value_indicator == 0)

print(f"The number of zeros in the 'UniqueValueIndicator' attribute is: {count_zeros}")


In [ ]:
# Not necessary

with lp.connect("16_head.loom") as ds:
    # Get the gene expression data
    gene_expression_data = ds[:, :]
    
    # Get the values of the column with index 32
    column_values = gene_expression_data[12330:12399, :]
    
    # Print the values of the column
    print(column_values)

## 8. printing specific columns values (with indices)

In [ ]:
with lp.connect("16_head.loom") as ds:
    # Get the gene expression data
    gene_expression_data = ds[:, :]
    
    # Get the values of the column with index 32
    column_values = gene_expression_data[12330:12350, :]
    
    # Print the values of the column
    print(column_values)

# Adding n_counts to col attributes

## 1. Adding n_counts

In [ ]:
# adds them instantly to the existing dataset

In [ ]:
with lp.connect("16_head.loom") as ds:
    # Create a mask for values equal to -1
    mask = (ds[:, :] != -1)
    
    # Calculate the sum of expression values for each cell (column) while ignoring -1 values
    cell_sums = (ds[:, :] * mask).sum(axis=0)
    
    # Add the sum as a column attribute 'n_counts' to the Loom file
    ds.ca['n_counts'] = cell_sums

## 2. Double checking n_counts

### 1. First 10 samples

In [ ]:
with lp.connect("16_head.loom") as ds:
    # Access the 'n_counts' attribute for the first 10 columns (cells)
    n_counts_first_10 = ds.ca.n_counts[:10]

print(n_counts_first_10)

### 2. First row n_counts in loom using np

In [ ]:
with lp.connect("3_head.loom") as ds:
    # Get the gene expression data as a numpy array
    gene_expression_data = ds[:, :]

    # Calculate the sum of values in the first column
    sum_first_column = gene_expression_data[:, 0].sum()

print("Sum of values in the first column:", sum_first_column)

### 3. First rows in csv n_count

In [ ]:
df = pd.read_csv('5_sorted_head.csv')

# Calculate the sum of values in the first row and ignore NaNs
sum_first_row1 = pd.to_numeric(df.iloc[0, 16:], errors='coerce').sum()
sum_first_row2 = pd.to_numeric(df.iloc[1, 16:], errors='coerce').sum()
sum_first_row3 = pd.to_numeric(df.iloc[2, 16:], errors='coerce').sum()
sum_first_row4 = pd.to_numeric(df.iloc[3, 16:], errors='coerce').sum()
print("Sum of values in the first rows:", sum_first_row1, sum_first_row2, sum_first_row3, sum_first_row4)

# Special characters in genes' names

## 1. Checking of existance

In [ ]:
import csv

# List of common special characters to check for
special_characters = "!@#$%^&*()-=[]{}|;:'\",.<>?/"

# Initialize list to store indices and column names with special characters
indices_list = []
column_names_list = []

# Read only the header of the CSV file
with open('5_sorted_head.csv', 'r') as f:
    reader = csv.reader(f)
    header = next(reader)

    for idx, column_name in enumerate(header):
        if any(char in special_characters for char in column_name):
            indices_list.append(str(idx))
            column_names_list.append(column_name)

# Write the indices and column names to the output file
with open('7.5_char_in_gene_names.txt', 'w') as f:
    # Write indices
    f.write(','.join(indices_list))
    f.write('\n')
    
    # Write column names
    for name in column_names_list:
        f.write(name)
        f.write('\n')

## 2. Chuncking gene names in 450 for each txt (for biomart)

In [ ]:
# to be written

## 3. Biomart check (Manual)

## 4. Comparison of genes in dataset with biomart genes (Ensemble IDs)

In [ ]:
# Using biomart - genes in chuncks of 500
# ran on my own PC
# First extract the genes names with special characters, chucks them and then check with biomart
# code needs edit to export names in exclude.txt

# Initialize an empty list to store the gene names
all_gene_names = []

# Loop through the seven text files
for i in range(0, 7):
    df = pd.read_csv(f"{i}.txt", delimiter=',', header=0)  # Using the first row as header
    all_gene_names.extend(df['Gene name'].tolist())  # Extract the 'Gene name' column

# Convert the combined gene names to a set for easier comparison
all_gene_names_set = set(all_gene_names)

# Read the other text file and convert its content into a set
with open("7.5_char_total.txt", "r") as file:
    text_file_set = set(file.read().splitlines())

# Find the strings not shared between the two sets
#not_shared_in_text_file = all_gene_names_set - text_file_set
not_shared_in_other_file = text_file_set - all_gene_names_set
print(len(not_shared_in_other_file))
#print("Strings not shared in the text file:", not_shared_in_text_file)
print("Strings not shared in the other file:", not_shared_in_other_file)

## 5. Masking gene names with special char (adding attr) from loom file

In [ ]:
# Be applied on the complete dataset
import loompy
import numpy as np

# Read the txt file to get the list of genes to exclude
with open('exclude_genes.txt', 'r') as file:
    exclude_genes = file.read().replace("'", "").split(", ")
    exclude_genes = set(exclude_genes)  # Convert list to set for efficient lookup

# Connect to the loom file
with loompy.connect('your_file.loom') as ds:
    # Get the gene names from the loom file
    gene_names = ds.ra.gene  # Assuming the row attribute for gene names is "gene"

    # Initialize a numpy array to store new row attributes
    new_row_attribute = np.ones(ds.shape[0], dtype=int)  # Initialize with ones

    # Loop through gene names and update the row attribute
    for i, gene in enumerate(gene_names):
        if gene in exclude_genes:
            new_row_attribute[i] = 0  # Set to 0 if the gene is in the exclude list

    # Add the new row attribute to the loom file
    ds.ra['ExcludeFlag'] = new_row_attribute


# TDigest test practice

In [ ]:
another_digest = TDigest()
another_digest.batch_update(i for i in range(100))
another_digest.update(0.55)
print(another_digest.percentile(15))

In [ ]:
median_digests = TDigest()
print(median_digests)
median_digests.batch_update(i for i in range(5))
print(median_digests)
median_digests.update(1)
print(median_digests)
print(median_digests.percentile(50))

In [ ]:
row_digests = [TDigest() for i in range(3)]
print(row_digests)
row_digests[0].update(50,20) # 20 of 50s it means
print(row_digests)
print(row_digests[0].percentile(50))

# NON-ZERO MEDIAN

## 1. Real median value of the first row

In [ ]:
with lp.connect("16_head.loom") as ds:
    # Get the gene expression data
    gene_expression_data = ds[:, :]
    n_counts_values = ds.ca.n_counts
    
    # Get the values of the first row
    first_row_values0 = gene_expression_data[0, :]
    first_row_values1 = gene_expression_data[1, :]
    first_row_values2 = gene_expression_data[2, :]
    
    normalized_values0 = first_row_values0 / n_counts_values
    normalized_values1 = first_row_values1 / n_counts_values
    normalized_values2 = first_row_values2 / n_counts_values
    
    # Calculate the median
    median_value0 = np.median(normalized_values0) * 10000
    median_value1 = np.median(normalized_values1) * 10000
    median_value2 = np.median(normalized_values2) * 10000
    
    # Print the median
    print("Median of the rows:", median_value0, median_value1, median_value2)

## 2. Checking median for few rows

In [ ]:
input_file = "16_head.loom"
output_file = input_file.replace(".loom", ".gene_median_digest_dict.pickle")

with lp.connect(f"{input_file}") as data:
    # define coordinates of protein-coding or miRNA genes
    coding_genes = data.ra.gene
    # Rows(genes)
    total_rows = data.shape[0]
    print(total_rows)
    #Cols(samples)
    print(data.shape[1])

    # Initiate tdigest
    median_digests = [TDigest() for _ in range(3)]  # only the first 10 coding genes
    # print the last one for checking later

    # initiate progress meters
    last_view_row = 0

    
    for (ix, selection, view) in data.scan(items=np.array(range(3)), axis=0):  # only the first 10 rows

        # subview = view

        # normalize by total counts per cell and multiply by 10,000 to allocate bits to precision
        subview_norm_array = view[:,:]/view.ca.n_counts*10000
        #print(subview_norm_array)
        
        if np.issubdtype(subview_norm_array.dtype, np.integer):
            subview_norm_array = subview_norm_array.astype(np.float32)
            
        # mask zeroes from distribution tdigest by filling with nan
        nonzero_data0 = np.ma.masked_equal(subview_norm_array, 0.0).filled(np.nan)
        nonzero_data = np.ma.masked_equal(nonzero_data0, -1).filled(np.nan)

        # update tdigestto review your data preprocessing steps
        for i in range(nonzero_data.shape[0]):
            median_digests[i+last_view_row].batch_update(nonzero_data[i,:])

        # update progress meters
        last_view_row = last_view_row + view.shape[0]

print(median_digests)
print(median_digests[0].percentile(50))
print(median_digests[1].percentile(50))
print(median_digests[2].percentile(50))

## 3. Non-zero median value computation (prints)

In [ ]:
input_file = "16_head.loom"
output_file = input_file.replace(".loom", ".gene_median_digest_dict.pickle")

with lp.connect(f"{input_file}") as data:
    # define coordinates of protein-coding or miRNA genes
    coding_genes = data.ra.gene
    # Rows(genes)
    total_rows = data.shape[0]
    print(total_rows)
    #Cols(samples)
    print(data.shape[1])

    # Initiate tdigest
    median_digests = [TDigest() for _ in range(len(coding_genes))]
    # print the last one for checking later

    # initiate progress meters
    last_view_row = 0

    
    for (ix, selection, view) in data.scan(items=np.array(range(total_rows)), axis=0):
        # subview = view

        # normalize by total counts per cell and multiply by 10,000 to allocate bits to precision
        subview_norm_array = view[:,:]/view.ca.n_counts*10000
        #print(subview_norm_array)
        
        if np.issubdtype(subview_norm_array.dtype, np.integer):
            subview_norm_array = subview_norm_array.astype(np.float32)
            
        # mask zeroes from distribution tdigest by filling with nan
        nonzero_data0 = np.ma.masked_equal(subview_norm_array, 0.0).filled(np.nan)
        nonzero_data = np.ma.masked_equal(nonzero_data0, -1).filled(np.nan)

        # update tdigestto review your data preprocessing steps
        for i in range(nonzero_data.shape[0]):
            median_digests[i+last_view_row].batch_update(nonzero_data[i,:])

        # update progress meters
        last_view_row = last_view_row + view.shape[0]

print(median_digests)

In [ ]:
# NEEDS TO BE EXPORTED AS A PICKLE FILE CONTAINING DICTIONARY